In [ ]:
from pathlib import Path
import ibis

# 查找 pyproject.toml 文件，来确定项目根目录
current_dir = Path(__name__).resolve().parent
project_dir = current_dir
while project_dir != project_dir.parent:
    if (project_dir / "pyproject.toml").exists():
        break
    project_dir = project_dir.parent
else:
    raise FileNotFoundError("Could not find pyproject.toml in any parent directory")

DB_PATH = project_dir / 'data/metadata.db'
print(DB_PATH)
con = ibis.duckdb.connect(database=DB_PATH, read_only=True)

# 查看所有表
print(con.list_tables())

cash_flow = con.table('cash_flow')
company_cash_flow = cash_flow.filter(cash_flow.ts_code == '600519.SZ')
print(company_cash_flow.compile())
company_cash_flow.execute()


/Users/yapex/workspace/stock_downloader/data/metadata.db
['balance_sheet', 'cash_flow', 'daily_basic', 'income_statement', 'stock_basic', 'stock_daily', 'trade_cal']


,ts_code,ann_date,f_ann_date,end_date,comp_type,report_type,end_type,net_profit,finan_exp,c_fr_sale_sg,...,net_cash_rece_sec,credit_impa_loss,use_right_asset_dep,oth_loss_asset,end_bal_cash,beg_bal_cash,end_bal_cash_equ,beg_bal_cash_equ,update_flag,year
